In [36]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time

# Set up the WebDriver using Service
service = Service('/Users/sri_amirdha_sudha/Downloads/chromedriver_arm64/chromedriver')

# Initialize the WebDriver with the service
driver = webdriver.Chrome(service=service)

# Base URL simplified to control pagination with pageSize and page
base_url = 'https://www.audible.com/search?pageSize=50&page='

# Initialize an empty list to store product data
product_list = []

# Loop through pages (e.g., from 1 to 5; adjust as needed)
for page_num in range(1, 26):  # Adjust range to cover all pages you want to scrape
    # Construct the URL for each page by incrementing the page number
    url = base_url + str(page_num)
    driver.get(url)  # Open the URL

    # Wait for the page to load
    time.sleep(2)  # Adjust the sleep time if needed

    # Scrape data from the current page
    products = driver.find_elements(By.XPATH, '//li[contains(@class,"productListItem")]')
    
    # Extract details for each product
    for product in products:
        try:
            title = product.find_element(By.XPATH, './/h3[contains(@class,"bc-heading")]').text
        except NoSuchElementException:
            title = "No title available"

        try:
            author = product.find_element(By.XPATH, './/li[contains(@class,"authorLabel")]').text
        except NoSuchElementException:
            author = "No author available"

        try:
            length = product.find_element(By.XPATH, './/li[contains(@class,"runtimeLabel")]').text
        except NoSuchElementException:
            length = "No duration available"

        try:
            narrator = product.find_element(By.XPATH, './/li[contains(@class,"narratorLabel")]').text
        except NoSuchElementException:
            narrator = "No narrator available"

        try:
            language = product.find_element(By.XPATH, './/li[contains(@class,"languageLabel")]').text
        except NoSuchElementException:
            language = "No language available"

        try:
            rating = product.find_element(By.XPATH, './/li[contains(@class, "ratingsLabel")]//span[contains(@class, "bc-pub-offscreen")]').text
        except NoSuchElementException:
            rating = "No rating available"

        try:
            people = product.find_element(By.XPATH, './/li[contains(@class, "ratingsLabel")]//span[contains(@class, "bc-size-small")]').text
        except NoSuchElementException:
            people = "No people available"

        try:
            price = product.find_element(By.XPATH, './/p[contains(@id,"buybox-regular-price")]').text
        except NoSuchElementException:
            price = "No price available"

        # Append the product details as a dictionary to the list
        product_list.append({
            "Title": title,
            "Author": author,
            "Length": length,
            "Narrator": narrator,
            "Language": language,
            "Rating": rating,
            "People": people,
            "Price": price
        })

# Close the driver
driver.quit()

# Print the list of product data
for product in product_list:
    print(product)
    print("-" * 40)


{'Title': 'Pucking Sweet', 'Author': 'By: Emily Rath', 'Length': 'Length: 19 hrs and 44 mins', 'Narrator': 'Narrated by: Desiree Ketchum, Sean Crisden, Alex Kydd', 'Language': 'Language: English', 'Rating': 'No rating available', 'People': 'Not rated yet', 'Price': 'Regular price: $24.49'}
----------------------------------------
{'Title': 'Enticing Her Love: Sutton Steele', 'Author': 'By: Melissa Foster', 'Length': 'Length: 13 hrs and 2 mins', 'Narrator': 'Narrated by: Savannah Peachwood, Jacob Morgan', 'Language': 'Language: English', 'Rating': '5 out of 5 stars', 'People': '3 ratings', 'Price': 'Regular price: $24.95'}
----------------------------------------
{'Title': 'Knight of Shadows', 'Author': 'By: Kel Kade', 'Length': 'Length: 14 hrs and 54 mins', 'Narrator': 'Narrated by: Nick Podehl', 'Language': 'Language: English', 'Rating': 'No rating available', 'People': 'Not rated yet', 'Price': 'Regular price: $54.99'}
----------------------------------------
{'Title': 'Celestial Nex

In [37]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

# Create a DataFrame from the list of dictionaries (your scraped data)
audible_table = pd.DataFrame(product_list)

# Display the first few rows to verify data
print(audible_table.head())

# Path to your credentials JSON file. Credential to connect to my JSON file
credentials_path = '/Users/sri_amirdha_sudha/Documents/Kaggle/project/marketpulse-435722-d790e39cbaec.json'

# Set up Google Sheets API credentials with necessary scopes
credentials = Credentials.from_service_account_file(
    credentials_path,
    scopes=[
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive'
    ]
)

# Authorize and connect to Google Sheets
gc = gspread.authorize(credentials)

# Open the specific Google Sheet by name
spreadsheet = gc.open('marketPulse')  # Replace with your actual Google Sheet name

# Select the first worksheet/tab
worksheet = spreadsheet.get_worksheet(0)  # This accesses the first sheet

# Clear existing data in the worksheet
worksheet.clear()

# Update the worksheet with DataFrame data
worksheet.update([audible_table.columns.values.tolist()] + audible_table.values.tolist())  # Updates the sheet with headers and data

print("Data has been updated in Google Sheets.")


                              Title              Author  \
0                     Pucking Sweet      By: Emily Rath   
1  Enticing Her Love: Sutton Steele  By: Melissa Foster   
2                 Knight of Shadows        By: Kel Kade   
3                   Celestial Nexus    By: JJ Bookerson   
4             From Hoping to Having     By: Julie Poole   

                       Length  \
0  Length: 19 hrs and 44 mins   
1   Length: 13 hrs and 2 mins   
2  Length: 14 hrs and 54 mins   
3  Length: 15 hrs and 26 mins   
4   Length: 8 hrs and 28 mins   

                                            Narrator           Language  \
0  Narrated by: Desiree Ketchum, Sean Crisden, Al...  Language: English   
1      Narrated by: Savannah Peachwood, Jacob Morgan  Language: English   
2                           Narrated by: Nick Podehl  Language: English   
3        Narrated by: Anne Marie Lebeaur, Jack Smith  Language: English   
4                           Narrated by: Julie Poole  Language: English

In [38]:
audible_table

,Title,Author,Length,Narrator,Language,Rating,People,Price
0,Pucking Sweet,By: Emily Rath,Length: 19 hrs and 44 mins,"Narrated by: Desiree Ketchum, Sean Crisden, Al...",Language: English,No rating available,Not rated yet,Regular price: $24.49
1,Enticing Her Love: Sutton Steele,By: Melissa Foster,Length: 13 hrs and 2 mins,"Narrated by: Savannah Peachwood, Jacob Morgan",Language: English,5 out of 5 stars,3 ratings,Regular price: $24.95
2,Knight of Shadows,By: Kel Kade,Length: 14 hrs and 54 mins,Narrated by: Nick Podehl,Language: English,No rating available,Not rated yet,Regular price: $54.99
3,Celestial Nexus,By: JJ Bookerson,Length: 15 hrs and 26 mins,"Narrated by: Anne Marie Lebeaur, Jack Smith",Language: English,5 out of 5 stars,6 ratings,Regular price: $54.99
4,From Hoping to Having,By: Julie Poole,Length: 8 hrs and 28 mins,Narrated by: Julie Poole,Language: English,No rating available,Not rated yet,Regular price: $19.95
...,...,...,...,...,...,...,...,...
1245,Nature’s Symphony of Relaxing Birdsongs in Tun...,By: Cedar Skye,Length: 7 hrs and 2 mins,Narrated by: Cedar Skye,Language: English,5 out of 5 stars,19 ratings,Regular price: $19.95
1246,It,By: Stephen King,Length: 44 hrs and 55 mins,Narrated by: Steven Weber,Language: English,4.5 out of 5 stars,"61,568 ratings",Regular price: $29.99
1247,Never Lie,By: Freida McFadden,Length: 7 hrs and 21 mins,Narrated by: Leslie Howard,Language: English,4.5 out of 5 stars,"9,823 ratings",Regular price: $27.29
1248,Leonard Cohen: The Man Who Saw the Angels Fall,By: Christophe Lebold,Length: 19 hrs and 38 mins,Narrated by: Vlasta Vrana,Language: English,3 out of 5 stars,1 rating,Regular price: $25.00
